In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy import stats
from scipy import sparse
import scipy
from collections import defaultdict
import implicit
from implicit.als import AlternatingLeastSquares
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVDpp
import rectorch
from rectorch.models.baseline import SLIM, Random, Popularity
from rectorch.models.mf import EASE
from rectorch.data import DataProcessing
from rectorch.samplers import ArrayDummySampler, SparseDummySampler
from rectorch.evaluation import evaluate
from rectorch.utils import collect_results, prepare_for_prediction

# Data Parsing

In [11]:
# initialize data
item_threshold = 1 # used to filter out user/artist pairs that have been 
                   #listened to less than the threshold number of times
popular_artist_fraction = 0.2 # top cutoff for what we consider popular artists, in this case the top 20%

user_events_file = 'data/user_events.txt'
low_user_file = 'data/low_main_users.txt'
medium_user_file = 'data/medium_main_users.txt'
high_user_file = 'data/high_main_users.txt'

In [12]:
#read in user events file
cols = ['user', 'artist', 'album', 'track', 'timestamp']
df_events = pd.read_csv(user_events_file, sep='\t', names=cols)
print('No. of user events: ' + str(len(df_events)))
df_events.head() # check it is all read in properly

No. of user events: 28718087


,user,artist,album,track,timestamp
0,31435741,2,4,4,1385212958
1,31435741,2,4,4,1385212642
2,31435741,2,4,4,1385212325
3,31435741,2,4,4,1385209508
4,31435741,2,4,4,1385209191


## User-Artist Matrix

In [13]:
# create unique user-artist matrix
df_events = df_events.groupby(['user', 'artist']).size().reset_index(name='listens')
print('No. user-artist pairs: ' + str(len(df_events)))
# each row contains a unique user-artist pair, along with how many times the
# user has listened to the artist
#df_events.sort_values(by=['user'])

No. user-artist pairs: 1755361


In [14]:
df_events.head(1400)

,user,artist,listens
0,1021445,12,43
1,1021445,16,1
2,1021445,28,7
3,1021445,29,1
4,1021445,46,1
...,...,...,...
1395,1021445,1864220,3
1396,1021445,1864221,4
1397,1021445,1864222,1
1398,1045479,3,9


In [15]:
# filters out artist/user pairs who havent been listened two more than
# item_threshold amount of times to reduce potential load
# kept to 1 currently, so we dont filter out any data
df_events = df_events[df_events['listens'] >= item_threshold] 

# With 1, we see no difference between user-artist pairs here
print('No. filtered user-artist pairs: ' + str(len(df_events))) 

# here, we see the number of unique artists in our matrix
print('No. unique artists: ' + str(len(df_events['artist'].unique())))

No. filtered user-artist pairs: 1755361
No. unique artists: 352805


#### How many artists have users listened to?

In [16]:
# get matrix where each row is a user-id and how many artists they've 
#listened to
user_dist = df_events['user'].value_counts() 

# counts how many unique users there are. prints out user id & a count of how 
# many rows they're included in, which effectively shows how many artists 
# they listen to
num_users = len(user_dist)
print('Mean artists of all users: ' + str(user_dist.mean()))
print('Min artists of all users: ' + str(user_dist.min()))
print('Max artists of all users: ' + str(user_dist.max()))

user_dist.head()

Mean artists of all users: 585.1203333333333
Min artists of all users: 18
Max artists of all users: 4011


41888522    4011
4393555     3700
40029632    3678
26874346    3544
29736410    3529
Name: user, dtype: int64

#### How many users listen to an artist?

In [17]:
# get artist distribution
# same as previous but with artists, shows artist-id and how many times they
# were listened to buy unique users
artist_dist = df_events['artist'].value_counts()
num_artists = len(artist_dist)
print('No. artists: ' + str(num_artists))
df_events['artist'].value_counts().head

No. artists: 352805


<bound method NDFrame.head of 135        1389
1602       1359
46         1325
320        1297
27         1290
           ... 
3124286       1
1029181       1
1023032       1
1008679       1
3087545       1
Name: artist, Length: 352805, dtype: int64>

In [18]:
# get number of  popular artists
num_top_artists = int(popular_artist_fraction * num_artists)

# getting the top top_fraction (0.2) percent of artists, so finding how many
# artists make up 20% of total artists, and then only using the artists those
#number of the most popular aritsts
top_artist_dist = artist_dist[:num_top_artists]
print('No. top artists: ' + str(len(top_artist_dist)))

No. top artists: 70561


In [19]:
# read in users
# user file is just user_id and their mainstreaminess value 
low_users = pd.read_csv(low_user_file, sep=',').set_index('user_id')
medium_users = pd.read_csv(medium_user_file, sep=',').set_index('user_id')
high_users = pd.read_csv(high_user_file, sep=',').set_index('user_id')
num_users = len(low_users) + len(medium_users) + len(high_users)
print('Num users: ' + str(num_users))

Num users: 3000


## Getting Users From Each Popularity Group & Their 10 Most Listened To Artists 

### (For Analysis of Streaming Service Algorithmic Bias)

In [20]:
toList = df_events.loc[df_events['user'] == 42845367].sort_values(by=['listens'], ascending=False)
toList.head() #grabbing random users top 10 artists in 1 of the 3 groups

,user,artist,listens
1513514,42845367,495,205
1513639,42845367,15624,201
1513657,42845367,27107,171
1513483,42845367,163,156
1513485,42845367,172,156


In [21]:
to_list_2 = toList['artist'].tolist()[:20]
print(to_list_2)

[495, 15624, 27107, 163, 172, 4311, 8047, 7, 56, 245, 52741, 173416, 140, 1703, 3887, 42794, 1092, 2558, 54, 42835]


# Calculating GAP of User Profiles

In [22]:
# placeholder vars for numerator of GAPp, waiting to be divided by sie of group
low_gap_p = 0
medium_gap_p = 0
high_gap_p = 0
total_gap_p = 0
#Count for sanity check
low_count = 0
med_count = 0
high_count = 0

for u, df in df_events.groupby('user'):
    
    no_user_artists = len(set(df['artist'])) # profile size //number of artists in users profile
    # get popularity (= fraction of users interacted with item) of user items and calculate average of it
    user_pop_artist_fraq = sum(artist_dist[df['artist']] / num_users) / no_user_artists 
    
    if u in low_users.index: # get user group-specific values
        low_gap_p += user_pop_artist_fraq
        low_count += 1
    elif u in medium_users.index:
        medium_gap_p += user_pop_artist_fraq
        med_count += 1
    else:
        high_gap_p += user_pop_artist_fraq
        high_count += 1

total_gap_p = (low_gap_p + medium_gap_p + high_gap_p) / num_users
low_gap_p /= len(low_users) # average popularity of items/artists in low/med/high groups (gap = group average popularity)
medium_gap_p /= len(medium_users)
high_gap_p /= len(high_users)
print('Low count (for check): ' + str(low_count))
print('Med count (for check): ' + str(med_count))
print('High count (for check): ' + str(high_count))
print(low_gap_p)
print(medium_gap_p)
print(high_gap_p)

Low count (for check): 1000
Med count (for check): 1000
High count (for check): 1000
0.04963328792099549
0.054371119359489226
0.06286028679778642


### Min-Max Scaling Ratings (Not Using Right Now)

In [ ]:
### Scale listening counts on a scale from 1-1000
"""scaled_df_events = pd.DataFrame()
for user_id, group in df_events.groupby('user'):
    #print(group)
    min_listens = group['listens'].min()
    max_listens = group['listens'].max()
    std = (group['listens'] - min_listens) / (max_listens - min_listens)
    scaled_listens = std * 999 + 1
    to_replace = group.copy()
    to_replace['listens'] = scaled_listens
    #print(to_replace)
    scaled_df_events = scaled_df_events.append(to_replace)
scaled_df_events.head()  """ 
#df_events.groupby('user').head()

# Analysis of Alternating Least Squares with the Implicit Library

### Shaping the Data

In [64]:
#Normalize data to compare artist popularity vs recommendations
#Normalize artist popularity
normalized_artist_dist = pd.DataFrame(artist_dist)
normalized_artist_dist.columns = ['listens']
normalized_artist_dist['listens'] /= num_users
normalized_artist_dist.head()

num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0
#num_times_recommended.head()
#normalized_artist_dist.head()
#For potential use with graphs, not being used right now

In [65]:
def user_events_file_to_lil_matrix():
    # Artist to User matrix where artist_user_matrix[a, u] = num of times user u listened to artist a

    # 352805, 3000 (total artists, users)
    rows, cols = 352805, 3000
    artist_user_matrix = scipy.sparse.lil_matrix((rows, cols), dtype=int)

    # user	artist	album	track	timestamp

    user_dict = {}  # simplify user id to 1, 2, 3 ...
    artist_dict = {}

    # populate with user_events_file
    with open("/home/jimi/music_fairness/LFM-1b-surprise-and-implicit-analysis/ML_Base_Project/data/user_events.txt", 'r') as fp:
        line = fp.readline()
        loop_count = 0
        while line:
            # get data from line
            line = fp.readline()
            parts = line.split("\t")

            # end case
            try:
                user_id = int(parts[0])
                artist_id = int(parts[1])
            except ValueError:
                print("end of file " + line)
                break

            # use user_dict to shorten user_id
            if user_id not in user_dict:
                # this user_id has not bee seen
                user_dict[user_id] = len(user_dict)
            user_idx = user_dict[user_id]

            # use track_dict to shorten track_id
            if artist_id not in artist_dict:
                # this user_id has not bee seen
                artist_dict[artist_id] = len(artist_dict)
            artist_idx = artist_dict[artist_id]

            # increment count of user to track
            artist_user_matrix[artist_idx, user_idx] += 1

            # progress marker
            loop_count = loop_count + 1
            if loop_count % 10000000 == 0:
                print(str(loop_count) + "/ 28718087")  # / num of lines in file

    print(len(user_dict))
    print(len(artist_dict))

    # helpful dicts for converting artist and user count back to their ids
    user_count_to_id_dict = {v: k for k, v in user_dict.items()}
    artist_count_to_id_dict = {v: k for k, v in artist_dict.items()}

    return artist_user_matrix, user_dict, artist_dict, user_count_to_id_dict, artist_count_to_id_dict


## FUNCTIONS FROM from jmsteinw's blog post ## a_u_tuple


In [42]:
def make_train(ratings, pct_test=0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings,
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix.

    parameters:

    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix.

    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the
    training set for later comparison to the test set, which contains all of the original ratings.

    returns:

    training_set - The altered version of the original data with a certain percentage of the user-item pairs
    that originally had interaction set back to zero.

    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order
    compares with the actual interactions.

    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy()  # Make a copy of the original set to be the test set.
    test_set[test_set != 0] = 1  # Store the test set as a binary preference matrix
    training_set = ratings.copy()  # Make a copy of the original data we can alter as our training set.
    nonzero_inds = training_set.nonzero()  # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))  # Zip these pairs together of user,item index into list
    random.seed(0)  # Set the random seed to zero for reproducibility
    num_samples = int(
        np.ceil(pct_test * len(nonzero_pairs)))  # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples)  # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples]  # Get the user row indices
    item_inds = [index[1] for index in samples]  # Get the item column indices
    training_set[user_inds, item_inds] = 0  # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros()  # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds))  # Output the unique list of user rows that were altered


# Usage
# a_to_u_train, a_to_u_test, altered_users = make_train(csr_sparse_matrix, pct_test = 0.2)

In [43]:
a_u_tuple = user_events_file_to_lil_matrix()
a_u_matrix = a_u_tuple[0] #artist/user matrix
back_to_user_id = a_u_tuple[3] #dictionary to convert smaller user ids to original
back_to_artist_id = a_u_tuple[4]# same above but for artists
print("done")

10000000/ 28718087
20000000/ 28718087
end of file 
3000
352805
done


In [44]:
u_to_a_train, u_to_a_test, altered_users = make_train(a_u_matrix.T.tocsr(), pct_test=0.2) #makes training/test set
print("done")

done


### Training the Algorithm

In [45]:
# split original matrix into user vector and artist vector through ALS
user_vecs, artists_vecs = implicit.alternating_least_squares((u_to_a_train).astype('double'),iterations=50, factors=50)
print("done")

[15:35:08-140421]  This method is deprecated. Please use the AlternatingLeastSquares class instead
[15:35:08-140421]  GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.
[15:35:08-140421]  OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading


  0%|          | 0/50 [00:00<?, ?it/s]

done


In [46]:
print(np.array(user_vecs[0,:].dot(artists_vecs.T))[:10]) # gets first 5 recommendations

[0.7734145  0.9261155  0.50536424 0.83280516 0.9921204  0.84349346
 0.86699915 0.72439003 0.8522642  0.24458975]


### Get Top N Recommendations

In [47]:
def get_top_artists_implicit(user_vecs, artists_vecs, mf_train, back_to_user_id,back_to_artist_id, num=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        user_vecs: the feature vector for users
        mf_train: training matrix
        artists_vecs: the feature vector for artists
        back_to_user_id: dictionary to convert back to original user id
        back_to_artist_id: dictionary to convert back to original artist id
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id), ...] of size n.
    """
    top_artists_dict = defaultdict(list)
        # creating dictionary where user id is the key, and the val is a list of tuples of the artist id and 
        # the rating it thinks the user would give it
        
    for user_id in range(len(user_vecs)):
        
        pref_vec = mf_train[user_id,:].toarray() # Get the ratings from the training set ratings matrix
        pref_vec = pref_vec.reshape(-1)
        pref_vec =  pref_vec + 1 # Add 1 to everything, so that artists not listened to yet are equal to 1
        pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
        
        rec_vector = user_vecs[user_id,:].dot(artists_vecs.T) # Get dot product of user vector and all artist vectors
        # Scale this recommendation vector between 0 and 1
        
        min_max = MinMaxScaler()
        rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
        recommend_vector = pref_vec*rec_vector_scaled 
        # Items already purchased have their recommendation multiplied by zero
        
        product_idx = np.argsort(recommend_vector)[::-1][:num] # Sort the indices of the items into order 
        # of best recommendations
        rec_list = [] # start empty list to store items
        actual_user_id = back_to_user_id[user_id]
        #print(actual_user_id)
        
        for index in product_idx:
            actual_artist_id = back_to_artist_id[index]
            rec_list.append(actual_artist_id)
            
        top_artists_dict[actual_user_id] = rec_list
    print("done")
    return top_artists_dict
    
    

In [48]:
implixit_top_n = get_top_artists_implicit(user_vecs, artists_vecs, u_to_a_train,back_to_user_id, back_to_artist_id,num = 10) 
#gets top n artists for all the users in the dataset
print("done")

done
done


In [49]:
print(implicit_top_n[6532902]) #testing out getting top recommendations for a user

[2908, 15483, 6443, 43, 2836, 1998, 93, 3103, 6236, 2562]


In [50]:
print(artist_dist[6443]) #getting popularity of artist by indexing by id

180


### Calculating GAPr and Delta GAP for ALS

In [52]:
# Train and test the four algorithms on our data set
low_gap_r_list = []
medium_gap_r_list = []
high_gap_r_list = []
total_gap_r_list= []
#keeps track of the GAPr of each of the algs

alg_recommendations = [] #Keeps track of how many times each artist was recommended

#Will put for loop here with training, but for now we already have predictions
    
num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0 #resets for each algorithm
    
    
# resets to 0 before calculating for new alg
low_gap_r = 0
medium_gap_r = 0
high_gap_r = 0
    
#keeps track of group size since it is unpredictable since the test set is selected randomly
num_low_users = 0
num_medium_users = 0
num_high_users = 0
    
#keeps track of the mean absolute error of the current alg
low_mae = 0
medium_mae = 0
high_mae = 0
top_artists = implicit_top_n #Gets the top N recommendations for each user
    

for user_id, ratings in top_artists.items():
    artist_id_list = [] #user profile size to compute top fraction in GAPr
    for artist_id in ratings:
            
        artist_id_list.append(artist_id)
        num_times_recommended.loc[artist_id] += 1 #increments the number of times the artist was recommended

    gap = sum(artist_dist[artist_id_list]/ num_users) / len(artist_id_list) #fraction in numerator for GAPr
    #print("gap:" ,gap)
    if user_id in low_users.index: #summation of all of the top fractions to compute numerator for GAPr
        low_gap_r += gap
        num_low_users += 1 #increments group size to get the denominator to compute GAPr

    elif user_id in medium_users.index:
        medium_gap_r += gap
        num_medium_users += 1

    elif user_id in high_users.index:
        high_gap_r += gap
        num_high_users += 1
alg_recommendations.append(num_times_recommended)
            
total_gap_r = (low_gap_r + medium_gap_r + high_gap_r) / (num_low_users + num_medium_users + num_high_users)
low_gap_r = low_gap_r / num_low_users #Computing GAPr for each group size for each algorithm
medium_gap_r = medium_gap_r / num_medium_users
high_gap_r = high_gap_r / num_high_users

total_gap_r_list.append(total_gap_r)
low_gap_r_list.append(low_gap_r)
medium_gap_r_list.append(medium_gap_r)
high_gap_r_list.append(high_gap_r)

print("done")

done


In [53]:
#Computes change in GAP for each RecSys alg
delta_gap_low_list = []
delta_gap_medium_list = []
delta_gap_high_list = []
delta_gap_total_list = []

for i in range(len(low_gap_r_list)):
    delta_gap_low = ((low_gap_r_list[i] - low_gap_p) / low_gap_p)
    delta_gap_medium = ((medium_gap_r_list[i] - medium_gap_p) / medium_gap_p)
    delta_gap_high = ((high_gap_r_list[i] - high_gap_p) / high_gap_p)
    delta_gap_total = ((total_gap_r_list[i] - total_gap_p) / total_gap_p)
    
    delta_gap_low_list.append(delta_gap_low)
    delta_gap_medium_list.append(delta_gap_medium)
    delta_gap_high_list.append(delta_gap_high)
    delta_gap_total_list.append(delta_gap_total)
    
print(delta_gap_low_list[0])

print(delta_gap_medium_list[0])

print(delta_gap_high_list[0])

print(delta_gap_total_list[0])


1.5132151940425258
1.7332807395549104
1.7789633524148147
1.6850323399738416


### Calculating AUC for ALS

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics.

    parameters:

    - predictions: your prediction output

    - test: the actual target result you are comparing to

    returns:

    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    # shuffle list of predictions (shuffle function)
    # shuffling dissassociates link to artist - > roc of .5 (random)
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered.

    parameters:

    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model

    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one.

    altered_users - The indices of the users where at least one user/item pair was altered from make_train function

    test_set - The test set constucted earlier from make_train function

    returns:

    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''

    store_auc = []  # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = []  # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis=0)).reshape(-1)  # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users:  # Iterate through each user that had an item altered
        training_row = training_set[user, :].toarray().reshape(-1)  # Get the training set row
        zero_inds = np.where(training_row == 0)  # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user, :]
        pred = user_vec.dot(item_vecs).toarray()[0, zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user, :].toarray()[0, zero_inds].reshape(-1)
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training
        pop = pop_items[zero_inds]  # Get the item popularity for our chosen items
        curr_auc_score = auc_score(pred, actual)
        store_auc.append(curr_auc_score)  # Calculate AUC for the given user and store
        curr_pop_score = auc_score(pop, actual)
        popularity_auc.append(curr_pop_score)  # Calculate AUC using most popular and score
        # print(user, "\t", curr_auc_score , "\t", curr_pop_score)
    # End users iteration

    return float('%.3f' % np.mean(store_auc)), float('%.3f' % np.mean(popularity_auc))
    # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [ ]:
rec_auc, pop_auc = calc_mean_auc(u_to_a_train, altered_users,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(artists_vecs.T)],
                                             u_to_a_test)
print(rec_auc)
print(pop_auc)